In [1]:
import numpy as np
import pandas as pd
import os, sys, json
import twint
import nest_asyncio

In [2]:
nest_asyncio.apply()

datapath = '../data/twitter_data'

In [3]:
def tweets2df(tweets, country):
    def tweet2json(tweet):
        return {
            'id': tweet.id,
            'datestamp': tweet.datestamp,
            'timestamp': tweet.timestamp,
            'username': tweet.username,
            'tweet': tweet.tweet,
            'replies_count': tweet.replies_count,
            'retweets_count': tweet.retweets_count,
            'likes_count': tweet.likes_count,
            'url': tweet.link,
            'country': country
        }
    
    jsons = [tweet2json(tweet) for tweet in tweets]
    df = pd.DataFrame(jsons)
    return df

In [4]:
def config_twint(username, since='2020-01-22'):
    twint.output.tweets_list = []
    tw = twint.Config()
    tw.Username = newspaper
    tw.Since = '2020-01-22'
    tw.Store_object = True
    tw.Stats = True
    tw.Hide_output = True
    return tw

In [10]:
def create(country, newspaper):
    """
    """
    print(f'Creating {newspaper} of {country}')
    dfname = f'{country}_{newspaper}.tsv'
    tw = config_twint(newspaper)

    twint.run.Search(tw)

    df = tweets2df(twint.output.tweets_list, country)
    if (dfname == 'China_chinaorgcn.tsv'):
        df.to_json(f'{datapath}/{dfname}')
    else:
        df.to_csv(f'{datapath}/{dfname}', sep='\t\t', index=False)

def update(country, newspaper):
    """
    """
    print(f'Updating {newspaper} of {country}')
    dfname = f'{country}_{newspaper}.tsv'
    past = pd.read_csv(f'{datapath}/{dfname}', sep='\t', engine='python')
    most_recent = past['datestamp'][0] + " " + past['timestamp'][0] 
    
    tw = config_twint(newspaper, most_recent)
    twint.run.Search(tw)

    df = tweets2df(twint.output.tweets_list, country)
    df = pd.concat([df, past], ignore_index=True)
    df.to_csv(f'{datapath}/{dfname}', sep='\t', index=False)

In [6]:
countries = {
    'US': ['cnn', 'nytimes', 'huffpost', 'FoxNews'],
    'Brazil': ['g1', 'folha', 'Estadao', 'JornalOGlobo', 'JornaldoBrasil'],
    'Italy': ['repubblica', 'Corriere', 'Libero_official', 'virgilio_it'],
    'UK': ['BBCNews', 'guardian', 'MailOnline', 'Telegraph'],
    'China': ['ChinaDaily', 'PDChina', 'shanghaidaily', 'chinaorgcn']
}

In [7]:
fields = ['id', 'datestamp, timestamp', 'username', 'tweet', 'replies_count', 'retweets_count', 'likes_count', 'url']
formatted = '"' + "\t".join(['{' + field + '}' for field in fields]) + '"'

In [11]:
for country in countries.keys():
    if (country in ['US', 'Brazil', 'Italy', 'UK']): continue
    newspapers = countries[country]
    for newspaper in newspapers:
        if newspaper != 'chinaorgcn': continue
        dfname = f'{country}_{newspaper}.tsv'
        if dfname in os.listdir(datapath) and False:
            update(country, newspaper)
        else:
            create(country, newspaper)
    
print('done')

Creating chinaorgcn of China


TypeError: "delimiter" must be a 1-character string

In [ ]:
"""
for country in countries.keys():
    newspapers = countries[country]
    for newspaper in newspapers:
        dfname = f'{country}_{newspaper}.tsv'
        df = pd.read_csv(f'{datapath}/{dfname}', sep='\t', engine='python')
        df.drop_duplicates(subset='id', keep=False, inplace=True)
        df.to_csv(f'{datapath}/{dfname}', sep='\t', index=False)
"""

In [17]:
df = tweets2df(twint.output.tweets_list, country)
df.to_json(f'{datapath}/{dfname}')

In [18]:
df.head()

,id,datestamp,timestamp,username,tweet,replies_count,retweets_count,likes_count,url,country
0,1244502032523771904,2020-03-30,02:49:40,chinaorgcn,[Video] Cloisonné - The 'Blue Jewel' in the ro...,1,1,8,https://twitter.com/chinaorgcn/status/12445020...,China
1,1244475851900465152,2020-03-30,01:05:38,chinaorgcn,Opinion: Political considerations behind US co...,2,2,10,https://twitter.com/chinaorgcn/status/12444758...,China
2,1244445883879456768,2020-03-29,23:06:34,chinaorgcn,Small businesses in Beijing seek to adapt amid...,2,6,32,https://twitter.com/chinaorgcn/status/12444458...,China
3,1244132958740475905,2020-03-29,02:23:06,chinaorgcn,Wuhan deserves to be on the tourist bucket lis...,67,40,175,https://twitter.com/chinaorgcn/status/12441329...,China
4,1244126977126105089,2020-03-29,01:59:20,chinaorgcn,Normal life in Wuhan gradually resumes https:...,9,21,67,https://twitter.com/chinaorgcn/status/12441269...,China


In [14]:
dfname

'China_chinaorgcn.tsv'